In [4]:
# compenency query is for fasc diam, nerv diam, and lateriality

In [2]:
import pandas as pd

from quantdb.ingest import Queries, InternalIds
from quantdb.client import get_session
from quantdb.ingest_tabular.models import (
    # Ingest,
    # Unit,
    Objects,
    DescriptorsQuant,
    ValuesQuant,
)

session = get_session()
# ingest = Ingest(session=session)

object = Objects(
    id="N:dataset:aa43eda8-b29a-4c25-9840-ecbd57598afc",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)

/var/folders/l0/pm3myv0s1k3gsmpf7vpbd59m0000gn/T/ipykernel_79997/2106051682.py:16: SAWarning: relationship 'ObjDescInst.values_cat' will copy column obj_desc_inst.object to column values_cat.object, which conflicts with relationship(s): 'Objects.values_cat' (copies objects.id to values_cat.object). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="values_cat"' to the 'ObjDescInst.values_cat' relationship. (Background on this warning at: https://sqlalche.me/e/20/qzyx) (This warning originated from the `configure_mappers()` process, which was invoked automatically in response to a user-initiated operation.)
  object = Objects(
/var/fol

In [6]:
# ingest.insert_objects(
#     objects=[
#         Objects(id="55c5b69c-a5b8-4881-a105-e4048af26fa5", id_type="dataset"),
#     ]
# )

df = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
df

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
NFasc,15,5,8,1,7,2,5,5,12,7,...,1,2,1,1,2,1,1,1,1,3
dFasc_um,"[437.21394287682614, 258.82574913412543, 307.9...","[120.12238286446609, 243.86502700380925, 322.2...","[365.4037755654016, 284.53752037921583, 53.536...",657.1827113056797,"[455.24233996428916, 509.9466847724346, 287.73...","[968.8828744830811, 204.86529698345961]","[219.6782211850974, 94.38546727436437, 550.442...","[1060.537068877718, 326.60460392259876, 310.04...","[337.6256234720049, 141.62779031797697, 532.80...","[143.42358541101993, 102.82738563544903, 157.5...",...,223.63951592404896,"[128.92615216318802, 32.1159656068769]",116.79797449421602,149.14465156614057,"[140.8971679287751, 146.24382498588818]",120.78887429872461,134.09449385025067,128.9177766574117,125.04421241450241,"[39.145387059629215, 105.77491277927702, 53.86..."
dNerve_um,2546.924091677267,2019.0831006899302,1615.1166896226846,1025.5346591258096,1838.3739820401397,1964.0189658353256,1864.19101371549,1882.4170241446818,2539.9242270389204,1522.4910816505858,...,236.65094413545614,215.72315387765067,126.85396089312266,154.73480397719217,321.23247394440426,128.24156004932206,146.56921053945464,137.13386043953284,132.3890001103472,219.5877970120584
laterality,L,L,L,L,L,L,L,L,L,A,...,L,A,A,A,A,A,A,A,A,A
level,C,C,C,C,C,C,C,C,C,A,...,C,A,A,A,A,A,A,A,A,A
sex,M,M,M,M,M,F,F,F,F,M,...,F,M,M,M,M,M,F,F,F,F
sub_sam,C50-2,C53-2,C54-3,C55-3,C57-3,C46-2,C47-2,C56-3,C58-3,C54-2,...,R18-1,R16-1,R17-1,R25-1,R23-1,R20-1,R21-1,R26-1,R24-1,R22-1
thkperi_um,"[24.794722747116708, 27.92527295706188, 17.016...","[23.91547828345668, 15.931285254737446, 29.499...","[19.700748058232335, 28.26263525939993, 9.6459...","[26.020421901161313, 46.18048430372056, 13.711...",6.81653463390343,6.505714105703589,"[3.7263428752987267, 4.669721606191388]","[2.3394073608833956, 2.4148193659335533, 3.437...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Testing values_quant for NFasc

In [7]:
# objects [x]
# descriptors_inst  # TODO []
# obj_desc_quant  # TODO []
# obj_desc_int # TODO []

# value
# object
# desc_inst
# desc_quant
# values_blob

object = Objects(
    id="N:dataset:aa43eda8-b29a-4c25-9840-ecbd57598afc",
    id_type="dataset",
    id_file=None,
    id_internal=None,
)
object.id

'aa43eda8-b29a-4c25-9840-ecbd57598afc'

# dream ingest

In [8]:
ingest.insert_descriptors_quant(
    descriptor_quants=[
        DescriptorQuant(
            label="fascicle cross section diameter um min",
            domain=26,  # "fiber-cross-section" # TODO: label to ID
            aspect=5,  # "diameter",  # TODO: label to ID
            unit=4,  #  pixel",  # TODO: label to ID
            aggregation_type="min",
        ),
    ]
)

# dream ingest
csv = pd.read_csv("data/CadaverVNMorphology_OutputMetrics.csv", index_col=0)
ingest.insert_descriptors_quant(**csv.to_dict(orient="records"))

NameError: name 'ingest' is not defined